# OpenGL helpers

> Helper functions to setup OpenGL for simple compute workloads

In [ ]:
#| default_exp gl_helpers

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import os
import time
import ctypes
import glob
import re

import numpy as np
from PIL import Image
import OpenGL.GLES3 as gl
from OpenGL.GL.shaders import compileProgram,compileShader

In [ ]:
#| export
os.environ['PAN_MESA_DEBUG'] = 'trace,sync'
os.system("rm -f pandecode.dump.*")
_egl = ctypes.cdll.LoadLibrary('./test-egl.so')
assert(_egl.setup_gl() == 0)

In [ ]:
#| export
_shaders = {}
def computeShader(src):
    shader = _shaders.get(src) or compileProgram(compileShader(src, gl.GL_COMPUTE_SHADER))
    gl.glUseProgram(shader)
    _shaders[src] = shader

In [ ]:
#| export
_textures = {}
def createTexture(w, h, texid=0, fmt=gl.GL_RGBA32F, output=True, src=None):
    global shader_format
    if fmt == gl.GL_RGBA32F:
        shader_format='rgba32f'
    elif fmt == gl.GL_RGBA16F:
        shader_format='rgba16f'
    else:
        shader_format="unsupported format: "+str(fmt)
    texture = _textures.get((texid, w, h, fmt, output))
    if not texture:
        texture = gl.glGenTextures(1)
        gl.glActiveTexture(gl.GL_TEXTURE0 + texid)
        gl.glBindTexture(gl.GL_TEXTURE_2D, texture)
        gl.glTexStorage2D(gl.GL_TEXTURE_2D, 1, fmt, w, h)
        _textures[(texid,w,h,fmt,output)] = texture
    if src is not None:
        gl.glActiveTexture(gl.GL_TEXTURE0 + texid)
        gl.glBindTexture(gl.GL_TEXTURE_2D, texture)
        gl.glTexSubImage2D(gl.GL_TEXTURE_2D, 0, 0, 0, w, h, gl.GL_RGBA, gl.GL_FLOAT, src)
    if output:
        gl.glBindImageTexture(texid, texture, 0, gl.GL_FALSE, 0, gl.GL_WRITE_ONLY, fmt)
    else:
        gl.glBindImageTexture(texid, texture, 0, gl.GL_FALSE, 0, gl.GL_READ_ONLY, fmt)
    return texture

In [ ]:
#| export
_fbo = gl.glGenFramebuffers(1);
def downloadTexture(texture, w, h):
    gl.glBindFramebuffer(gl.GL_FRAMEBUFFER, _fbo);
    gl.glFramebufferTexture2D(gl.GL_FRAMEBUFFER, gl.GL_COLOR_ATTACHMENT0, gl.GL_TEXTURE_2D, texture, 0);
    pixels = np.zeros((w,h,4), dtype=np.float32)
    gl.glReadPixels(0, 0, w, h, gl.GL_RGBA, gl.GL_FLOAT, pixels)
    gl.glBindFramebuffer(gl.GL_FRAMEBUFFER, 0);
    image = Image.frombytes('RGBA', (w, h), (pixels*255).astype(np.uint8))
    image = image.transpose(Image.Transpose.FLIP_TOP_BOTTOM)
    return image

def downloadTextureFloat(texture, w, h):
    gl.glBindFramebuffer(gl.GL_FRAMEBUFFER, _fbo);
    gl.glFramebufferTexture2D(gl.GL_FRAMEBUFFER, gl.GL_COLOR_ATTACHMENT0, gl.GL_TEXTURE_2D, texture, 0);
    pixels = np.zeros((w,h,4), dtype=np.float32)
    gl.glReadPixels(0, 0, w, h, gl.GL_RGBA, gl.GL_FLOAT, pixels)
    gl.glBindFramebuffer(gl.GL_FRAMEBUFFER, 0);
    return pixels

In [ ]:
#| export
def showLastShaderDisassembly():
    dumpfd = open(sorted(glob.glob('pandecode.dump.*'))[-1])
    dump = dumpfd.read()
    disasm = re.findall(r"(clause_.*?)(?=^\s*State:|\Z)", dump, flags=re.DOTALL|re.MULTILINE)[-1]
    FMAs = len(re.findall(r'^    \*FMA', disasm, flags=re.MULTILINE))
    FMAslots = len(re.findall(r'^    \*', disasm, flags=re.MULTILINE))
    print(f"FMAs: {FMAs / FMAslots * 100:.2f}% ({FMAs} / {FMAslots})")
    print()
    print(disasm)
    
def showShaderDisassembly(source):
    computeShader(source)
    gl.glDispatchCompute(1, 1, 1)
    showLastShaderDisassembly()

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()